In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c test-recsys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p "/content/drive/My Drive/sbermarket"

In [ ]:
from zipfile import ZipFile
import os

for filename in os.listdir('.'):
    if filename.endswith(".zip"):
        with ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('drive/My Drive/sbermarket')
    else:
        continue

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

import tqdm
from itertools import chain
import seaborn as sns

from collections import defaultdict
from typing import List

from zipfile import ZipFile
import os

from torch import nn
from torch.nn import functional as F
from torch.optim import Adam

from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math

import json
import pickle
from pathlib import Path

WORKING_DIR = 'drive/My Drive/sbermarket'

columns = ['price', 'quantity', 'discount', 'product_name', 'prod_idx', 'par_idx', 'mast_idx']

MAX_NPROD = 50

### Ищем репрезентативных пользователей

In [ ]:
df = pd.DataFrame()

for filename in sorted(os.listdir('drive/My Drive/sbermarket/')):
    if filename.startswith('tab_2_products') and filename.endswith('csv'):
        print(filename)
        new_df = pd.read_csv('drive/My Drive/sbermarket/'+filename)
        new_df = pd.DataFrame(new_df.groupby('user_id')['order_id'].nunique())

        df = df.add(new_df, fill_value=0)

In [ ]:
repr_users = df[df['order_id'] > 5][df['order_id'] < 15].reset_index().drop('order_id', axis=1)
repr_users.to_csv('drive/My Drive/sbermarket/repr_users.csv')
print(repr_users.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(77368, 1)


the dataset will consist of 77368 users

In [ ]:
repr_users = pd.read_csv('drive/My Drive/sbermarket/repr_users.csv')

### Filter tables

In [ ]:
dfs = []
for filename in tqdm.tqdm(sorted(os.listdir('drive/My Drive/sbermarket'))):
    if filename.startswith('tab_2_products') and filename.endswith('csv'):
        print(filename)
        df = pd.read_csv('drive/My Drive/sbermarket/'+filename)
        df = df.set_index('user_id')
        df = df.loc[list(set(df.index).intersection(set(repr_users['user_id'])))]
        df = df.reset_index()
        dfs.append(df)


In [ ]:
products_df = pd.concat(dfs)
products_df.dropna(subset=['product_name', 'master_category_id'], inplace=True)

for col, new_col in zip(['product_id', 'parent_category_id', 'master_category_id'],
                        ['prod_idx', 'par_idx', 'mast_idx']):
    products_df[col] = products_df[col].astype('category')
    products_df[new_col] = products_df[col].cat.codes

print(products_df.shape)

(19357908, 14)


In [ ]:
products_df.to_pickle('drive/My Drive/sbermarket/filtered_df.pckl')

In [ ]:
nproducts = len(products_df.product_id.unique())
ncategories = len(products_df.parent_category_id.unique())
ncategories2 = len(products_df.master_category_id.unique())
print(nproducts, ncategories, ncategories2)

prod2idx = {prod_id: idx for idx, prod_id in enumerate(products_df.product_id.unique())}
mast_cat2idx = {prod_id: idx for idx, prod_id in enumerate(products_df.master_category_id.unique())}
par_cat2idx = {prod_id: idx for idx, prod_id in enumerate(products_df.parent_category_id.unique())}

89931 119 608


In [ ]:
meta = {'nproducts' : nproducts, 'ncategories': ncategories, 'ncategories2': ncategories2, 
        'prod2idx': prod2idx,
        'mast_cat2idx': mast_cat2idx,
        'par_cat2idx': par_cat2idx}

with Path('drive/My Drive/sbermarket/meta.json').open('w') as f:
    json.dump(meta, f)

In [ ]:
products_df = pd.read_pickle('drive/My Drive/sbermarket/filtered_df.pckl')

In [ ]:
products_df.shape

(19357908, 14)

In [ ]:
with Path('drive/My Drive/sbermarket/meta.json').open('r') as f:
    meta = json.load(f)


nproducts = meta['nproducts']
ncategories = meta['ncategories']
ncategories2 = meta['ncategories2']
prod2idx = meta['prod2idx']
par_cat2idx = meta['par_cat2idx']
mast_cat2idx = meta['mast_cat2idx']

In [ ]:
import gc
gc.collect()

193

### PyTorch Dataset

In [ ]:
class MarketDataset(Dataset):
    def __init__(self, dataset, max_nprod=None, max_nord=None):
        self.df_dataset = dataset.set_index('user_id')
        self.user2idx = {u_id: i for i, u_id in enumerate(self.df_dataset.index.unique())}
        self.idx2user = {i: u_id for u_id, i in self.user2idx.items()}
        self.columns = ['price', 'quantity', 'discount', 'product_name', 'prod_idx', 'par_idx', 'mast_idx']
        self.max_nprod = float('inf') if max_nprod is None else max_nprod
        self.max_nord = max_nord
    
    def __len__(self):
        return len(self.user2idx)

    def __getitem__(self, user_idx):
        if user_idx < 2600:
            return []
        item = defaultdict(list)
        try:
            user_df = self.df_dataset.loc[self.idx2user[user_idx]]
            nprods = []
            orders_ids = list(user_df['order_id'])

            if self.max_nord is not None:
                max_nord = len(orders_ids)
            else:
                max_nord = max_nord

            line_ids = []
            for order in orders_ids[-max_nord:]:
                line_ids_ = user_df.query(f'order_id == {order}')['line_item_id']
                line_ids_ = line_ids_.sample(min(self.max_nprod, len(line_ids_)))
                products_from_order = user_df[user_df['line_item_id'].isin(line_ids_)]
                for c in columns:
                    item[c].append(products_from_order[c].values.tolist())
        except:
             return self.__getitem__(user_idx+1)

        return item

In [ ]:
from torch.nn.utils.rnn import pad_sequence

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def collate_fn(batch):
    # encode sentences
    sentences = []
    items_without_name = defaultdict(list)
    try:
        for b in batch:
            for c in columns:
                if c != 'product_name':
                    tensor_per_batch = pad_sequence(
                        [torch.Tensor(x) for x in b[c]], batch_first=True # for each order in sample
                    ).to(device) #
                    items_without_name[c].append(tensor_per_batch)

        # items_without_name['product_name'] = sentences
        for k, v in items_without_name.items():
            items_without_name[k] = torch.stack(v, 0) # batch dim
        return items_without_name
    except:
        return []

In [ ]:
import gc
gc.collect()

523

### Model

In [ ]:
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens
        in the sequence. The positional encodings have the same dimension as
        the embeddings, so that the two can be summed. Here, we use sine and cosine
        functions of different frequencies.
    .. math::
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
        

In [ ]:
class TransformerRec(nn.Module):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    def __init__(self, nproducts, ncategories1, ncategories2, nhead, nhid, nlayers, dropout=0.5):
        super().__init__()

        self.price_emb = nn.Linear(1, 1)
        self.quantity_emb = nn.Linear(1, 1)
        self.discount_emb = nn.Linear(1, 1)

        self.product_id_emb = nn.Embedding(nproducts, 201)
        self.parent_id_emb = nn.Embedding(ncategories1, 100)
        self.master_id_emb = nn.Embedding(ncategories2, 100)

        self.ninp = 3 + 201 + 1 * 100 

        self.src_mask = None
        self.pos_encoder = PositionalEncoding(self.ninp, dropout)
        encoder_layers = TransformerEncoderLayer(self.ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        
        self.decoder = nn.Linear(self.ninp, nproducts)
        self.device = 'cuda' if torch.cuda.is_available else 'cpu'

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, has_mask=True):
        tr_len = src['prod_idx'].shape[1]
        if has_mask:
            device = src['prod_idx'].device
            if self.src_mask is None or self.src_mask.size(0) != tr_len:
                mask = self._generate_square_subsequent_mask(tr_len).to(device)
                self.src_mask = mask
        else:
            self.src_mask = None
        src_ = {}
        for k, v in src.items():
            if not k.endswith('idx') and k != 'product_name': 
                src_[k] = v.reshape(list(v.shape) + [1])
            else:
                src_[k] = v

        src_ = torch.cat([
                          # src_['product_name'],
                          self.price_emb(src_['price'].float()), 
                          self.quantity_emb(src_['quantity'].float()),
                          self.discount_emb(src_['discount'].float()), 
                          self.product_id_emb(src_['prod_idx'].long()),
                          self.parent_id_emb(src_['par_idx'].long()),
                          #self.master_id_emb(src_['mast_idx'].long()),
                          ], -1)
        mask = (src['prod_idx'] == 0)
        src_[mask] = 0.0
        src_ = (src_.sum(2) / ((~mask).sum(2)+1e-5).unsqueeze(-1).repeat(1,1,src_.shape[-1])).permute(1, 0, 2)

        #src_ = src_ * math.sqrt(self.ninp)
        #src = self.pos_encoder(src_)
        output = self.transformer_encoder(src_, self.src_mask)
        output = self.decoder(output)
        return output.permute(1, 0, 2)


In [ ]:
def get_input_target(batch, device):
    return {k:v[:, :-1, :].to(device) for k, v in batch.items()}, \
    batch['prod_idx'][:, 1:, :].to(device)

In [ ]:
def loss_fn(pred, target):
    pred = pred[0].repeat(target.shape[-1], 1)
    mask = (target == 0)
    target = target[0].reshape(-1)
    mask = mask[0].reshape(-1) 
    loss = F.cross_entropy(pred, target, reduction='none')
    mask = target == 0
    loss[mask] = 0.0
    return loss.mean()

### Euristics: consider only those products that user has already bought earlier

Very useful to get higher metric

In [ ]:
def filter_pred(pred, inp_prods):
    hist = torch.zeros(pred.shape[-1]).to(pred.device)
    for p in inp_prods.squeeze():
        hist[p] = 1
    return pred * hist

In [ ]:
def apk(actual, predicted, k=10):
    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(true, predicted, k=10):
    '''
    true, predicted: tensors
    '''
    predicted = torch.topk(predicted, k)[1]
    return np.mean([apk(a,p,k) for a,p in zip(true.tolist(), predicted.tolist())])

In [ ]:
n_epoch = 10
n_heads = 4
n_layers = 3
n_hid = 128
dropout = 0.2
lr = 1e-3
model_file_name = 'model3.pt'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = TransformerRec(nproducts, ncategories, ncategories2, n_heads,  n_hid, n_layers, dropout)
_ = model.to(device)


In [ ]:
optimizer = Adam(model.parameters(), lr=lr)

In [ ]:
market_dataset = MarketDataset(products_df)
trainloader = DataLoader(market_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

#### Train loop

In [ ]:
if hasattr(tqdm.tqdm, '_instaces'):
    tqdm.tqdm._instances.clear()

if hasattr(tqdm.trange, '_instaces'):
    tqdm.trange._instances.clear()

model.train()
for ep in range(n_epoch):
    ep_loss = 0
    map_ep = 0
    batch_id = 0
    for batch in tqdm.tqdm(trainloader):
        batch_id += 1
        input, target = get_input_target(batch, device)
        pred = model(input)
        pred = filter_pred(pred, input['prod_idx'].long())
        loss = loss_fn(pred, target.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        map_ep += mapk(target[0], pred[0], MAX_NPROD)
        ep_loss += loss.item()

        if batch_id % 100 == 0:
            print(f'\tIter {batch_id}, epoch loss: {ep_loss/ batch_id}, map_ep: {map_ep/ batch_id}')
        if batch_id % 1000 == 0:
            torch.save(
                {
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'loss': ep_loss/ batch_id,
                    'map': map_ep/ batch_id
                }, Path(WORKING_DIR, model_file_name)
            )

    print(f'MAP: {map_ep / len(trainloader)}')
    print(f'Epoch {ep+1}, loss: {ep_loss / len(trainloader)}')

  0%|          | 101/77361 [00:11<2:19:44,  9.21it/s]

	Iter 100, epoch loss: 6.231178970336914, map_ep: 0.06675325912523791


  0%|          | 202/77361 [00:22<2:15:08,  9.52it/s]

	Iter 200, epoch loss: 5.790545735359192, map_ep: 0.07126948063865615


  0%|          | 301/77361 [00:33<2:23:06,  8.97it/s]

...

	Iter 9900, epoch loss: 3.9238367747538017, map_ep: 0.06931144880632328


 13%|█▎        | 9998/77361 [19:09<2:05:09,  8.97it/s]

	Iter 10000, epoch loss: 3.9205549569904803, map_ep: 0.0692687413278583


 13%|█▎        | 10062/77361 [19:19<2:02:03,  9.19it/s]

KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load(Path(WORKING_DIR, model_file_name))['model'])
_ = model.eval()

In [ ]:
submission_df = pd.read_pickle(Path(WORKING_DIR, 'submission1_df.pckl'))

In [ ]:
gc.collect()

0

In [ ]:
test_market_dataset = MarketDataset(submission_df, max_nord=5, max_nprod=20)

In [ ]:
test_loader = DataLoader(test_market_dataset, batch_size=1, collate_fn=collate_fn, shuffle=False)

In [ ]:
idx2prod = {idx:prod_id for prod_id, idx in prod2idx.items()}

In [ ]:
with Path('subm.pkl').open('rb') as f:
    preds = pickle.load(f)

In [ ]:
#preds = {'Id': [], 'Predicted': []}
idx2user = test_market_dataset.idx2user
with torch.no_grad():
    for batch_id, batch in enumerate(test_loader):
        if batch_id < 2600:
            continue
        if batch_id % 100 == 0:
            print(batch_id)
        if batch_id % 10000 == 0:
            with Path('subm.pkl').open('wb') as f:
                pickle.dump(preds, f)
        input = {k:v[:, :-1, :].to(device) for k, v in batch.items()}
        if input['prod_idx'].shape[1] == 0:
            preds['Id'].append(idx2user[batch_id])
            preds['Predicted'].append([int(idx2prod[p]) for p in pred])
        else:
            pred = model(input)
            pred = filter_pred(pred, input['prod_idx'].long())
            pred = pred[0, -1, :].topk(50)[1].tolist()
            preds['Id'].append(idx2user[batch_id])
            preds['Predicted'].append(' '.join([idx2prod[p] for p in pred]))


2600


In [ ]:
pd.to_csv('submission.csv')